# Load Required Libraries and Model
Import necessary libraries and load the base model and PEFT adapter if specified. Set up tokenizer and model configuration for chat.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# Import necessary libraries
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel  # Import PEFT for LoRA adapters

# Define paths for base model and PEFT model adapter
base_model_path = "model_ckpt/DeepSeek-R1-Distill-Qwen-1.5B"
peft_model_path = "trained_models/codealpaca_lora_20250221_030756_r8/checkpoint-60000"

# Load the tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
model = AutoModelForCausalLM.from_pretrained(base_model_path)

# Load the PEFT model if specified
if peft_model_path:
    print(f"Loading PEFT model from {peft_model_path}")
    model = PeftModel.from_pretrained(model, peft_model_path)
    model = model.merge_and_unload()

# Set up the text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading PEFT model from trained_models/codealpaca_lora_20250221_030756_r8/checkpoint-60000


Device set to use cuda:0


# Create Chat Interface
Define helper functions to format user input and model responses, including proper prompt templates and response parsing.

In [3]:
# Define a function to format the user input into a prompt
def format_prompt(user_input):
    return f"User: {user_input}\nAI:"

# Define a function to parse the model's response
def parse_response(response):
    return response.split("AI:")[1].strip()

# Define a function to generate a response from the model
def generate_response(user_input):
    prompt = format_prompt(user_input)
    generated = generator(prompt, max_length=1024,truncation=True, num_return_sequences=1)[0]["generated_text"]
    return parse_response(generated)

# Example usage
user_input = "Hello, who are you?"
response = generate_response(user_input)
print(response)

Hi there! I'm an AI, and I'm here to provide information and help with tasks. I'm here to answer questions, give you insights, and offer useful resources. Is there something specific you'd like to know or explore? I'm here to help! Hi there! I'm an AI, and I'm here to provide information and help with tasks. I'm here to answer questions, give you insights, and offer useful resources. Is there something specific you'd like to know or explore? I'm here to help! Hi there! I'm an AI, and I'm here to provide information and help with tasks. I'm here to answer questions, give you insights, and offer useful resources. Is there something specific you'd like to know or explore? I'm here to help! Hi there! I'm an AI, and I'm here to provide information and help with tasks. I'm here to answer questions, give you insights, and offer useful resources. Is there something specific you'd like to know or explore? I'm here to help! Hi there! I'm an AI, and I'm here to provide information and help with t

# Handle Model Generation
Create a function to generate model responses using the pipeline, including parameters like max_length, temperature, and top_p.

In [4]:
# Handle Model Generation

# Define a function to generate model responses using the pipeline
def generate_model_response(prompt, max_length=1024, temperature=0.7, top_p=0.9):
    """
    Generate a response from the model using the provided prompt and parameters.

    Args:
    - prompt (str): The input prompt for the model.
    - max_length (int): The maximum length of the generated response.
    - temperature (float): The sampling temperature.
    - top_p (float): The cumulative probability for nucleus sampling.

    Returns:
    - str: The generated response from the model.
    """
    response = generator(
        prompt,
        max_length=max_length,
        temperature=temperature,
        truncation=True,
        top_p=top_p,
        num_return_sequences=1
    )[0]["generated_text"]
    return response

# Example usage
user_input = "Tell me a joke about programming."
formatted_prompt = format_prompt(user_input)
response = generate_model_response(formatted_prompt)
parsed_response = parse_response(response)
print(parsed_response)

The algorithm is a machine that is written to carry out a specific task. It is a set of instructions that a computer or a machine can execute. The instructions are designed to perform a specific job and are usually written in a programming language. Programming languages are used to give instructions to a machine in order to carry out a specific task. Algorithms are the fundamental component of programming languages and are used to define the logic of the program. A program is only a collection of algorithms and instructions that are given to the machine. The machine will execute the algorithms and instructions given to it in order to achieve its purpose. Programming is an essential part of the job of a computer or a machine. It allows it to do its job and carry out complex tasks. It also allows it to automate and speed up processes. Programming can be both a source of complex logic and a source of errors if not handled properly. It is important to have a good understanding of the subj

# Interactive Chat Loop (Created by Claude Sonet 3.5)
Implement an interactive chat interface using IPython widgets or display, allowing users to input prompts and receive model responses.

In [5]:
# Interactive Chat Loop

from IPython.display import display, HTML
import ipywidgets as widgets

# Define a function to handle user input and display the chat
def chat_with_model(user_input):
    response = generate_response(user_input)
    chat_log.value += f"<b>User:</b> {user_input}<br><b>AI:</b> {response}<br><br>"

# Create a text area widget for chat log
chat_log = widgets.HTML(value="", placeholder="Chat log will appear here...", description="Chat Log:")

# Create a text box widget for user input
user_input = widgets.Text(placeholder="Type your message here...", description="Your Input:")

# Create a button widget to submit user input
submit_button = widgets.Button(description="Send")

# Define the button click event handler
def on_button_click(b):
    chat_with_model(user_input.value)
    user_input.value = ""

# Attach the event handler to the button
submit_button.on_click(on_button_click)

# Display the widgets
display(chat_log)
display(user_input)
display(submit_button)

HTML(value='', description='Chat Log:', placeholder='Chat log will appear here...')

Text(value='', description='Your Input:', placeholder='Type your message here...')

Button(description='Send', style=ButtonStyle())